In [25]:
import ccxt
import time
import pprint
import pandas as pd
import SecretConfig
import ta
from IPython.display import clear_output
import asyncio



In [26]:
async def fetch_data():

    global full_rolling_dataframe
    global top_10_values

    full_rolling_dataframe = pd.DataFrame()

    # Parcourir chaque contrat à terme
    for ticker_name in tickers_name_list:

        databyparams = pd.DataFrame()

        # Récupérer les informations pertinentes pour chaque contrat à terme
       
        databyparams[ticker_name + ' Last Price'] = data[ticker_name + ' Last Price'] 
        databyparams[ticker_name + ' Var Last Price'] = (data[ticker_name + ' Last Price'] - data[ticker_name + ' Last Price'].shift(wind_Var_Price)) /data[ticker_name + ' Last Price'].shift(wind_Var_Price) * 100
        databyparams[ticker_name + ' Var Carre Last Price'] = databyparams[ticker_name + ' Var Last Price'] ** 2
        databyparams[ticker_name + ' EMA_1'] = ta.trend.ema_indicator(data[ticker_name + ' Last Price'], window=wind_EMA1)
        databyparams[ticker_name + ' Var EMA_1'] = (databyparams[ticker_name + ' EMA_1'] - databyparams[ticker_name + ' EMA_1'].shift(wind_Var_EMA1)) / databyparams[ticker_name + ' EMA_1'].shift(wind_Var_EMA1) * 100
        databyparams[ticker_name + ' EMA_2'] = ta.trend.ema_indicator(data[ticker_name + ' Last Price'], window=wind_EMA2)
        databyparams[ticker_name + ' Var EMA_2'] = (databyparams[ticker_name + ' EMA_2'] - databyparams[ticker_name + ' EMA_2'].shift(wind_Var_EMA2)) / databyparams[ticker_name + ' EMA_2'].shift(wind_Var_EMA2) * 100
        databyparams[ticker_name + ' Diff Carre Price_EMA_1']  = (databyparams[ticker_name + ' Last Price'] - databyparams[ticker_name + ' EMA_1']) ** 2
        databyparams[ticker_name + ' Divergence Price_EMA_1'] = (databyparams[ticker_name + ' Diff Carre Price_EMA_1'] - databyparams[ticker_name + ' Diff Carre Price_EMA_1'].shift(wind_Div_Price_EMA_1)) / databyparams[ticker_name + ' Diff Carre Price_EMA_1'].shift(wind_Div_Price_EMA_1) * 100
        databyparams[ticker_name + ' Divergence Moyen Price_EMA_1'] = ta.trend.ema_indicator(databyparams[ticker_name + ' Divergence Price_EMA_1'], window=wind_Div_Moy_Price_EMA_1)
        databyparams[ticker_name + ' Diff Carre EMA_1_2']  = (databyparams[ticker_name + ' EMA_1'] - databyparams[ticker_name + ' EMA_2']) ** 2
        databyparams[ticker_name + ' Divergence EMA_1_2'] = (databyparams[ticker_name + ' Diff Carre EMA_1_2'] - databyparams[ticker_name + ' Diff Carre EMA_1_2'].shift(wind_Div_EMA1_2)) / databyparams[ticker_name + ' Diff Carre EMA_1_2'].shift(wind_Div_EMA1_2) * 100
        databyparams[ticker_name + ' Divergence Moyen EMA_1_2'] = ta.trend.ema_indicator(databyparams[ticker_name + ' Divergence EMA_1_2'], window=wind_Div_Moy_EMA1_2)
        databyparams[ticker_name + ' Condition'] = None

        full_rolling_dataframe = pd.concat([full_rolling_dataframe, databyparams], axis=1)

    full_rolling_dataframe = full_rolling_dataframe.copy()

    full_rolling_dataframe = full_rolling_dataframe.apply(pd.to_numeric)

    # pprint.pprint(datatouse)

    for ticker_name in tickers_name_list:
        
        full_rolling_dataframe[ticker_name + ' Condition'] = (full_rolling_dataframe[ticker_name + ' Divergence Moyen Price_EMA_1'] > 0) & (full_rolling_dataframe[ticker_name + ' Divergence Moyen EMA_1_2'] > 0)
        full_rolling_dataframe = full_rolling_dataframe.copy()

    top_10_variations = []
    # datatouse = datatouse.iloc[wind_EMA2 + wind_Div_Moy_EMA1_2:]
    # Parcourez chaque ligne de 'datatouse'
    for _, row in full_rolling_dataframe.iterrows():
        # Sélectionnez uniquement les colonnes qui se terminent par ' Var Carre Last Price'
        df_top_10_row = row.filter(regex=' Divergence Moyen Price_EMA_1$')
        data_top_10= pd.DataFrame(df_top_10_row)
        # print(data_fuck)
        column_name = data_top_10.columns[0]
        data_top_10[column_name] = pd.to_numeric(data_top_10[column_name], errors='coerce')
        top_10_values = data_top_10.nlargest(10, column_name)
        # print(top_10_values)

        # Obtenez les 10 plus grandes variations
        top_10 = top_10_values.index.tolist()

        # Supprimez ' Var Carre Last Price' de chaque nom de colonne
        top_10 = [name.replace(' Divergence Moyen Price_EMA_1', '') for name in top_10]

        # Ajoutez des None pour les lignes avec moins de 10 colonnes
        top_10 += [None] * (10 - len(top_10))

        # Ajoutez les résultats à la liste
        top_10_variations.append(top_10)

    # Créez un nouveau DataFrame avec seulement les 10 plus grandes variations
    top_10_values = pd.DataFrame(top_10_variations, columns=['top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8', 'top9', 'top10'], index=full_rolling_dataframe.index)

    # pprint.pprint(top_10_values)
    # pprint.pprint(rolling_occurence_count)


In [27]:
async def trad():
    
    global df_open_trade_info
    global occurences_count
    global engaged
    global frais
    global unclose_PnL
    global nbr_open_trade
    global balance
    global nbr_total_trade
    
    
    if not df_open_trade_info.empty:

        for index, row in df_open_trade_info.iterrows():

            if row["Status"] == "Open":
                
                if row["Side"] == "Long":

                    real_PnL = None
                    real_Size = None
                    df_open_trade_info["PnL"] = df_open_trade_info["PnL"].apply(pd.to_numeric)
                    df_open_trade_info["Size"] = df_open_trade_info["Size"].apply(pd.to_numeric)

                    if row["Trail stop price"] < full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price'] * (1 - trail_stop):

                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trail stop price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price'] * (1 - trail_stop)
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20+ x)], row["Symbol"] + ' Last Price']

                    elif row["Trail stop price"] >= full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price']:

                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Status"] = "Close"
                        trade_close_price = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20+ x)], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trade close price"] = trade_close_price
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Duration"]  = (pd.to_datetime(full_rolling_dataframe.index[(20 + x)]) - pd.to_datetime(row["Timestamp"])).round('S')
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"] = row["Size"] * (((trade_close_price* 1.0006) - row["Trade open price"]) / row["Trade open price"])
                        # df_open_trade_info["PnL"] = df_open_trade_info["PnL"].apply(pd.to_numeric)
                        real_PnL = df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"].item()
                        balance = balance + real_PnL
                        frais = frais + ((row["Size"] + real_PnL) * 0.0004) 
                        # print(f'trade {row["Symbol"]} close withe PnL {real_PnL}')
                        nbr_open_trade -= 1

                    else :
                        actual = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = actual
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"] = row["Size"] * (((actual* 1.0006) - row["Trade open price"]) / row["Trade open price"])

                        if nbr_open_trade > 25 and row["PnL"] <= 0 :
                          df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trail stop price"]  = row["Trail stop price"] + ((row["Actual price"] - row["Trail stop price"]) / 3)
                else:

                    if row["Trail stop price"] > full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price'] * (1 + trail_stop):

                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trail stop price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price'] * (1 + trail_stop)
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price']

                    elif row["Trail stop price"] <= full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price']:
                    
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Status"] = "Close"
                        trade_close_price = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trade close price"] = trade_close_price
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Duration"] = (pd.to_datetime(full_rolling_dataframe.index[(20 + x)]) - pd.to_datetime(row["Timestamp"])).round('S')
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"] = row["Size"] * -(((trade_close_price* 1.0006) - row["Trade open price"]) / row["Trade open price"])
                        # df_open_trade_info["PnL"] = df_open_trade_info["PnL"].apply(pd.to_numeric)
                        real_PnL = df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"].item()
                        balance = balance + real_PnL
                        frais = frais + ((row["Size"] + real_PnL) * 0.0004)
                        # print(f'trade {row["Symbol"]} close withe PnL {real_PnL}')
                        nbr_open_trade -= 1

                    else :
                        actual = full_rolling_dataframe.loc[full_rolling_dataframe.index[(20 + x)], row["Symbol"] + ' Last Price']
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Actual price"] = actual
                        df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "PnL"] = row["Size"] * -(((actual* 1.0006) - row["Trade open price"]) / row["Trade open price"])

                        if nbr_open_trade > 25 and row["PnL"] <= 0 :
                          df_open_trade_info.loc[df_open_trade_info["Symbol"] == row["Symbol"], "Trail stop price"]  = row["Trail stop price"] - ((row["Trail stop price"] - row["Actual price"]) / 3)


        engaged = df_open_trade_info.loc[df_open_trade_info['Status'] == 'Open', 'Size'].sum()
        unclose_PnL = df_open_trade_info.loc[df_open_trade_info['Status'] == 'Open', 'PnL'].sum()
        # balance = balance - (engaged * 0.9994)
        # df_open_trade_info = df_open_trade_info.tail(40)
        # await asyncio.sleep(0.005)
        # print(df_open_trade_info)
        
    
    for value in tradable:

        # pprint.pprint(f'for {value} condition is {full_rolling_dataframe[value + ' Condition'].iloc[(20 + x)]}')
        if full_rolling_dataframe[value + ' Condition'].iloc[(20 + x)] and balance > 0 and nbr_open_trade < 20 :

            trade_df = {}
            newtradedfrow = pd.DataFrame()
            
            if full_rolling_dataframe[value + ' Var Last Price'].iloc[(20 + x)] > 0 and full_rolling_dataframe[value + ' Last Price'].iloc[(20 + x)] > full_rolling_dataframe[value + ' EMA_1'].iloc[(20 + x)] > full_rolling_dataframe[value + ' EMA_2'].iloc[(20 + x)]:

                trade_df = {"Timestamp": full_rolling_dataframe.index[(20 + x)],
                "Symbol": value,
                "Side" : "Long",
                "Size" : round(balance * 0.05, 2),
                "Trade open price": full_rolling_dataframe.iloc[(20 + x)][value + ' Last Price'],
                "Actual price" : full_rolling_dataframe.iloc[(20 + x)][value + ' Last Price'],
                "Trail stop price" : full_rolling_dataframe.iloc[(20 + x)][value + ' Last Price'] * (1 - trail_stop),
                "Status" : "Open",
                "Trade close price" : None,
                "Duration" : None,
                "PnL" : None}

            elif full_rolling_dataframe[value + ' Var Last Price'].iloc[(20 + x)] < 0 and full_rolling_dataframe[value + ' Last Price'].iloc[(20 + x)] < full_rolling_dataframe[value + ' EMA_1'].iloc[(20 + x)] < full_rolling_dataframe[value + ' EMA_2'].iloc[(20 + x)]:

                trade_df = {"Timestamp": full_rolling_dataframe.index[(20 + x)],
                "Symbol": value,
                "Side" : "Short",
                "Size" : round(balance * 0.05, 2),
                "Trade open price": full_rolling_dataframe.iloc[(20 + x)][value + ' Last Price'],
                "Actual price" : full_rolling_dataframe.iloc[(20 + x)][value + ' Last Price'],
                "Trail stop price" : full_rolling_dataframe.iloc[(20 + x)][value + ' Last Price'] * (1 + trail_stop),
                "Status" : "Open",
                "Trade close price" : None,
                "Duration" : None,
                "PnL" : None}
            
            else:
                continue

            if df_open_trade_info.empty :
                df_open_trade_info = pd.DataFrame([trade_df])
                balance = balance - (trade_df["Size"] * 0.0004)
                frais = frais + (trade_df["Size"] * 0.0004)
                nbr_open_trade += 1
                nbr_total_trade += 1

            elif value not in df_open_trade_info["Symbol"].values: 
                newtradedfrow = pd.DataFrame([trade_df])
                df_open_trade_info["Size"] = df_open_trade_info["Size"].apply(pd.to_numeric, errors='coerce')
                if not newtradedfrow.isnull().all().all():
                    df_open_trade_info = pd.concat([df_open_trade_info, newtradedfrow])
                balance = balance - (trade_df["Size"] * 0.0004)
                frais = frais + (trade_df["Size"] * 0.0004)
                nbr_open_trade += 1
                nbr_total_trade += 1

        else:
            continue
            
        
        # clear_output(wait=True)
        # pprint.pprint(full_rolling_dataframe.tail(10))
    # pprint.pprint(df_open_trade_info)
    # pprint.pprint(f"nbr of open trade is {nbr_open_trade}")
    # pprint.pprint(f"Balance is now : {balance}")
    # pprint.pprint(f"Unrealised PnL is now : {unclose_PnL}")
    # pprint.pprint(f"Engage is now = {engaged}")
    # pprint.pprint(f"Cumulated frees is now = {frais}")
    
            # pprint.pprint(trade_result)
            # pprint.pprint(f"nbr of open trade is {nbr_open_trade}")

    # await asyncio.sleep(0.01)

In [ ]:
datafile = pd.read_csv("CSVTEST2_1000.csv")
data = datafile[['Timestamp'] + [col for col in datafile.columns if col.endswith(' Last Price') and 'Var' not in col]]
data = data.set_index('Timestamp')
tickers_name_list = [col.replace(' Last Price', '') for col in data.columns]

async def main():

    global wind_Var_Price 
    global wind_EMA1
    global wind_Var_EMA1 
    global wind_EMA2 
    global wind_Var_EMA2 
    global wind_Div_Price_EMA_1 
    global wind_Div_Moy_Price_EMA_1 
    global wind_Div_EMA1_2
    global wind_Div_Moy_EMA1_2 

    testnbr = 0
    final_result_df = pd.DataFrame()
    for trail_stop in [0.001, 0.0005]:
        for occur_min in range(3,6,1):
            for wind_Var_Price in range(3,5,1) :
                for wind_EMA1 in range(4,7,1) :
                    for wind_Var_EMA1 in range(3,5,1) :
                        for wind_EMA2 in range(7,10,1) :
                            for wind_Var_EMA2 in range (2,6,2):
                                for wind_Div_Price_EMA_1 in range (2,8,2):
                                    for wind_Div_Moy_Price_EMA_1 in range (2,6,2):
                                        for wind_Div_EMA1_2 in range (2,8,2) :
                                            for wind_Div_Moy_EMA1_2 in range (2,6,2) :
                                            
                                                global occurences_count
                                                global df_open_trade_info
                                                global tradable
                                                global x
                                                global balance
                                                global nbr_open_trade
                                                # global trail_stop
                                                global frais
                                                global nbr_total_trade
                                                global unclose_PnL
                                                global engaged
                                                          
                                                windparams = [wind_Var_Price, wind_EMA1, wind_Var_EMA1, wind_EMA2, wind_Var_EMA2, wind_Div_Price_EMA_1, wind_Div_Moy_Price_EMA_1, wind_Div_EMA1_2, wind_Div_Moy_EMA1_2]
                                                df_open_trade_info = pd.DataFrame()
                                                balance = 1500
                                                frais = 0
                                                engaged = 0
                                                # trail_stop = 0.001
                                                occurences_count = None
                                                nbr_open_trade = 0
                                                nbr_total_trade = 0
                                                unclose_PnL = 0
                                                pprint.pprint(windparams)
                                
                                                try:
                                                
                                                    await fetch_data()

                                                    x = 0

                                                    # pprint.pprint(full_rolling_dataframe)

                                                    for index, row in full_rolling_dataframe.iloc[20:].iterrows():

                                                        occurences_count = top_10_values.iloc[x: 20 + x].apply(pd.Series.value_counts)
                                                        rolling_occurence_count = pd.DataFrame(occurences_count)
                                                        rolling_occurence_count["Total_occur"] = rolling_occurence_count.sum(axis=1)
                                                        tradable = rolling_occurence_count[rolling_occurence_count["Total_occur"] > occur_min]
                                                        tradable = tradable.sort_values(by="Total_occur", ascending=False).index
                                                        # pprint.pprint(f'tradable is {tradable}')

                                                        await trad()

                                                        x += 1
                                            
                                                    pprint.pprint(df_open_trade_info)
                                                    pprint.pprint(f"nbr of open trade is {nbr_open_trade}")
                                                    pprint.pprint(f"Balance is now : {balance}")
                                                    pprint.pprint(f"Unrealised PnL is now : {unclose_PnL}")
                                                    pprint.pprint(f"Engage is now = {engaged}")
                                                    pprint.pprint(f"Cumulated frees is now = {frais}")
                                                    pprint.pprint(f"nbr of Total trade is {nbr_total_trade}")
                                                    pprint.pprint(windparams)
                                                    # display(full_rolling_dataframe)

                                                    config_result_df = pd.DataFrame([windparams + [occur_min, trail_stop, balance, unclose_PnL, frais, engaged, nbr_total_trade]],
                                                                                columns=['wind_Var_Price', 'wind_EMA1', 'wind_Var_EMA1', 'wind_EMA2',
                                                                                            'wind_Var_EMA2', 'wind_Div_Price_EMA_1', 'wind_Div_Moy_Price_EMA_1',
                                                                                            'wind_Div_EMA1_2', 'wind_Div_Moy_EMA1_2', 'occur_min', 'trail_stop', 'balance', 'unclose_PnL', 'frais', 'engaged', 'nbr_total_trade'])
                                                
                                                    final_result_df = pd.concat([final_result_df, config_result_df])

                                                    trade = "trade"
                                                    result = "result"
                                                    df_open_trade_info.to_csv(rf'C:\Users\IEL\Desktop\BitgetBotV1\Trade_backtest_Maxi\Trades_for_Result_row_{testnbr}.csv')
                                                    final_result_df.to_csv(rf'C:\Users\IEL\Desktop\BitgetBotV1\Result_backtest\Backtest_Maxi{result}.csv')
                                                    testnbr += 1
                                        
                                                except KeyboardInterrupt:
                                                    pprint.pprint("Interuption")


# Exécutez la fonction main
await main()